# Production economy: Workers and capitalists

**Table of contents**<a id='toc0_'></a>    
- 1. [Model](#toc1_)    
  - 1.1. [Parameters](#toc1_1_)    
  - 1.2. [Workers](#toc1_2_)    
  - 1.3. [Capitalists](#toc1_3_)    
  - 1.4. [Firm](#toc1_4_)    
  - 1.5. [Equilibrium](#toc1_5_)    
- 2. [Experiments](#toc2_)    
- 3. [Using a class](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
from types import SimpleNamespace

import numpy as np
from scipy import optimize

%load_ext autoreload
%autoreload 2

import ProductionEconomy

## 1. <a id='toc1_'></a>[Model](#toc0_)

Consider an economy consisting of $N_w$ **workers**, and $N_c$ **capitalists** and a single **firm** owned equally by the capitalists.

**Workers:** Consume, $c_w$, at a price $p$, and supply labor, $\ell_w$, at a wage of $w$. Maximize utility:
        
$$\max_{c_w\geq0,\ell_w\in[0,1]} \log (c_w+\kappa)- \omega \ell_w^{\eta} \text{ s.t } pc_w \leq w \ell_w,\,\,\,\omega,\kappa > 0, \eta \geq 1$$ 

Equivalently, substituting in the budget constraint with equality:

$$\max_{\ell_w\in[0,1]} \log \left( \frac{w \ell_w}{p}+\kappa \right)- \omega \ell_w^{\eta}$$ 

Denote ***optimal behavior*** $c_w^{\star}(p,w)$ and $\ell_w^{\star}(p,w)$.

**Capitalists:** Consume, $c_c$, at a price $p$, supply labor, $\ell_c$, at a wage $w$, and receives profits $\pi$. Maximize utility:
        
$$\max_{c_c\geq0,\ell_c\in[0,1]} \log (c_c+\kappa) - \omega \ell_c^{\eta} \text{ s.t } pc_c = w \ell_c + \pi, ,\,\,\,\omega,\kappa > 0, \eta \geq 1$$ 

Equivalently, substituting in the budget constraint with equality:

$$\max_{\ell_c\in[0,1]} \log \left( \frac{w \ell_c + \pi}{p}+\kappa \right)- \omega \ell_c^{\eta}$$ 

Denote ***optimal behavior*** $c_c^{\star}(p,w,\pi)$ and $\ell_c^{\star}(p,w,\pi)$.

**Firm:** Use the production function $f(\ell) = \ell^\alpha, \alpha \in (0,1)$. Maximize profits:

$$\max_{\ell\geq0} p f(\ell) - w\ell $$ 

Denote ***optional behavior*** by $\ell^{\star}(p,w)$. 

Implied ***production*** is $y^{\star}(p,w) = f(\ell^{\star}(p,w))$ and implied ***total profits*** are $\Pi^\star(p,w) = py^{\star}(p,w) - w\ell^{\star}(p,w)$ 

**Equilibrium:** A set of prices $(p,w)$ such that workers, capitalists and firms act optimally given prices and profit, and

1. **Goods market clears**: $N_w c_w^{\star}(p,w) + N_c c_c^{\star}(p,w,\pi) = y^\star(p,w)$
2. **Labor market clears**: $N_w \ell_w^{\star}(p,w) + N_c \ell_c^{\star}(p,w,\pi) = \ell^\star(p,w)$
3. **Profits received equal profits distributed**: $\pi = \frac{py^{\star}(p,w) - w\ell^{\star}(p,w)}{N_c}$

**Note I:** We can use $p=1$ as numeraire.

**Note II:** *Walras' Law* imply that if one of the markets clear, then the other one does too.

### 1.1. <a id='toc1_1_'></a>[Parameters](#toc0_)

Choose parameters:

In [2]:
par = SimpleNamespace()
par.kappa = 0.1
par.omega = 10
par.eta = 1.50
par.alpha = 0.50
par.Nw = 99
par.Nc = 1

### 1.2. <a id='toc1_2_'></a>[Workers](#toc0_)

In [3]:
def utility_w(c,l,par):
    """ utility of workers """
    
    return np.log(c+par.kappa)-par.omega*l**par.eta

def workers(p,w,par):
    """ maximize utility for workers """
    
    # a. solve
    obj = lambda l: -utility_w((w*l)/p,l,par)
    res = optimize.minimize_scalar(obj,bounds=(0,1),method='bounded')
    
    # b. save
    l_w_star = res.x
    c_w_star = (w*l_w_star)/p
    
    return c_w_star,l_w_star

**Small test:**

In [4]:
p = 1
for w in [0.5,1,1.5]:
    c,l = workers(p,w,par)
    print(f'w = {w:.2f} -> c = {c:.2f}, l = {l:.2f}')

w = 0.50 -> c = 0.03, l = 0.06
w = 1.00 -> c = 0.11, l = 0.11
w = 1.50 -> c = 0.18, l = 0.12


### 1.3. <a id='toc1_3_'></a>[Capitalists](#toc0_)

In [5]:
def utility_c(c,l,par):
    """ utility of capitalists """
    
    return np.log(c+par.kappa)-par.omega*l**par.eta

def capitalists(p,w,pi,par):
    """ maximize utility of capitalists """
    
    # a. solve
    obj = lambda l: -utility_c((w*l+pi)/p,l,par) # subsittute in the budget constraint
    res = optimize.minimize_scalar(obj,bounds=(0,1),method='bounded')
    
    # b. save
    l_c_star = res.x
    c_c_star = (w*l_c_star+pi)/p
    
    return c_c_star,l_c_star

**Small test:**

In [6]:
p = 1
pi = 0.1
for w in [0.5,1,1.5]:
    c,l = capitalists(p,w,pi,par)
    print(f'w = {w:.2f} -> c = {c:.2f}, l = {l:.2f}')

w = 0.50 -> c = 0.11, l = 0.02
w = 1.00 -> c = 0.16, l = 0.06
w = 1.50 -> c = 0.23, l = 0.09


**Question:** Any idea for another test?

### 1.4. <a id='toc1_4_'></a>[Firm](#toc0_)

In [7]:
def firm(p,w,par):
    """ maximize firm profits """
    
    # a. solve
    f = lambda l: l**par.alpha
    obj = lambda l: -(p*f(l)-w*l)
    x0 = [0.0]
    res = optimize.minimize(obj,x0,bounds=((0,None),),method='L-BFGS-B')
    
    # b. save
    l_star = res.x[0]
    y_star = f(l_star)
    Pi = p*y_star - w*l_star
    
    return y_star,l_star,Pi

**Small test:**

In [8]:
p = 1
for w in [0.5,1,1.5]:
    y,l,Pi = firm(p,w,par)
    print(f'w = {w:.2f} -> y = {y:.2f}, l = {l:.2f}, Pi = {Pi:.2f}')

w = 0.50 -> y = 1.00, l = 1.00, Pi = 0.50
w = 1.00 -> y = 0.50, l = 0.25, Pi = 0.25
w = 1.50 -> y = 0.33, l = 0.11, Pi = 0.17


### 1.5. <a id='toc1_5_'></a>[Equilibrium](#toc0_)

In [9]:
def evaluate_equilibrium(w,par,p=None,do_print=False):
    """ evaluate equilirium """
    
    # a. normalize output price
    p = 1 if p is None else p
    
    # b. optimal behavior of firm
    y_star,l_star,Pi = firm(p,w,par)
    pi = Pi/par.Nc
    
    # c. optimal behavior of households
    c_w_star,l_w_star = workers(p,w,par)
    c_c_star,l_c_star = capitalists(p,w,pi,par)
    
    # d. market clearing
    goods_mkt_clearing = par.Nw*c_w_star + par.Nc*c_c_star - y_star
    labor_mkt_clearing = par.Nw*l_w_star + par.Nc*l_c_star - l_star
    
    if do_print:
        
        u_w = utility_w(c_w_star,l_w_star,par)
        print(f'workers      : c = {c_w_star:6.4f}, l = {l_w_star:6.4f}, u = {u_w:7.4f}')
        u_c = utility_c(c_c_star,l_c_star,par)
        print(f'capitalists  : c = {c_c_star:6.4f}, l = {l_c_star:6.4f}, u = {u_c:7.4f}')        
        print(f'goods market : {goods_mkt_clearing:.8f}')
        print(f'labor market : {labor_mkt_clearing:.8f}')
        
    else:
    
        return goods_mkt_clearing


**Step 1:** Perform rough grid search to check when the goods market clears.

In [10]:
num_w = 10
grid_w = np.linspace(0.1,1.5,num_w)
grid_mkt_clearing = np.zeros(num_w)

for i,w in enumerate(grid_w):
    grid_mkt_clearing[i] = evaluate_equilibrium(w,par)
    print(f'w = {w:.2f} -> excess demand = {grid_mkt_clearing[i]:12.8f}')

w = 0.10 -> excess demand =  -2.45597063
w = 0.26 -> excess demand =  -0.33115179
w = 0.41 -> excess demand =   1.47824268
w = 0.57 -> excess demand =   3.60037473
w = 0.72 -> excess demand =   5.89988125
w = 0.88 -> excess demand =   8.29317023
w = 1.03 -> excess demand =  10.74049294
w = 1.19 -> excess demand =  13.22157416
w = 1.34 -> excess demand =  15.72439188
w = 1.50 -> excess demand =  18.24150339


**Step 2:** Find where *excess demand* changes sign - the equilibrium price must be within this range

In [11]:
left = np.max(grid_w[grid_mkt_clearing < 0])
right = np.min(grid_w[grid_mkt_clearing > 0])
print(f'equilibrium price must be in [{left:.2f},{right:.2f}]')

equilibrium price must be in [0.26,0.41]


**Step 3:** Use equation-solver / root-finder

In [12]:
res = optimize.root_scalar(evaluate_equilibrium,bracket=[left,right],method='bisect',args=(par,))
w = res.root
print(f'the equilibrium wage is {w:.4f}')

the equilibrium wage is 0.2864


**Show details:**

In [13]:
evaluate_equilibrium(w,par,do_print=True)

workers      : c = 0.0088, l = 0.0308, u = -2.2721
capitalists  : c = 0.8731, l = 0.0004, u = -0.0274
goods market : 0.00000004
labor market : 0.00000013


**Check I:** Does both markets clear?

**Check II:** Can we multiply both prices with the same factor? I.e. can we change the numeraire?

In [14]:
fac = 100
p_ = fac*1.0 
w_ = fac*w
evaluate_equilibrium(w_,par,p=p_,do_print=True)

workers      : c = 0.0088, l = 0.0308, u = -2.2721
capitalists  : c = 0.8731, l = 0.0004, u = -0.0274
goods market : -0.00000240
labor market : -0.00000840


## 2. <a id='toc2_'></a>[Experiments](#toc0_)

It is easy to extend this model in many directions: 

1. Should workers and capitalists have different tastes or producitvity?
1. Should there be government redistribution?
2. Other ideas?

## 3. <a id='toc3_'></a>[Using a class](#toc0_)

In [15]:
from ProductionEconomy import ProductionEconomyClass

**Look at `ProductionEconomy.py`:** Same code, but written as a class! 

In [16]:
model = ProductionEconomyClass()
print(model.par.kappa)

0.1


In [17]:
model.find_equilibrium()

grid search:
 w = 0.10 ->  -2.45597063
 w = 0.26 ->  -0.33115179
 w = 0.41 ->   1.47824268
 w = 0.57 ->   3.60037473
 w = 0.72 ->   5.89988125
 w = 0.88 ->   8.29317023
 w = 1.03 ->  10.74049294
 w = 1.19 ->  13.22157416
 w = 1.34 ->  15.72439188
 w = 1.50 ->  18.24150339

equilibrium price must be in [0.26,0.41]

the equilibrium wage is 0.2864

workers      : c = 0.0088, l = 0.0308, u = -2.2721
capitalists  : c = 0.8731, l = 0.0004, u = -0.0274
goods market : 0.00000004
labor market : 0.00000013


**Benefit I:** Fewer inputs and outputs, less risk of wrong ordering.

**Benefit II of class-based solution:** Easy access to all data.
E.g. capitalists share of total consumption.

In [18]:
C_w = model.par.Nw*model.sol.c_w_star
C_c = model.par.Nc*model.sol.c_c_star
print(f'capitalists share of total consumption is: {C_c/(C_c+C_w):.2f}')

capitalists share of total consumption is: 0.50


**Benefit III of class-based solution:** Easy to experiment with different parameters.

In [19]:
model.par.kappa = model.par.kappa/100 # lower kappa
model.find_equilibrium()

grid search:
 w = 0.10 ->  -0.93720172
 w = 0.26 ->   3.11578788
 w = 0.41 ->   6.01856761
 w = 0.57 ->   8.72067391
 w = 0.72 ->  11.35650742
 w = 0.88 ->  13.96702518
 w = 1.03 ->  16.56693675
 w = 1.19 ->  19.16044548
 w = 1.34 ->  21.74865061
 w = 1.50 ->  24.33227156

equilibrium price must be in [0.10,0.26]

the equilibrium wage is 0.1260

workers      : c = 0.0200, l = 0.1592, u = -4.4959
capitalists  : c = 1.9848, l = 0.0000, u =  0.6860
goods market : 0.00000037
labor market : 0.00000291


In [20]:
for k,v in model.sol.__dict__.items():
    print(f'{k:20s} = {v:6.2f}')

p                    =   1.00
w                    =   0.13
l_star               =  15.76
y_star               =   3.97
Pi                   =   1.98
pi                   =   1.98
l_w_star             =   0.16
c_w_star             =   0.02
l_c_star             =   0.00
c_c_star             =   1.98
goods_mkt_clearing   =   0.00
labor_mkt_clearing   =   0.00
